In [2]:
from SpellChecker import SpellChecker
from collections import Counter
import re

In [3]:
checker = SpellChecker()

In [4]:
def read_text(filename):
    with open(filename, "r") as file:
        return file.read()

In [5]:
text_correct = read_text("text_correct.txt").lower()
text_misspel = read_text("text_misspel.txt").lower()

In [6]:
# raw_words = re.findall(r'\w+', raw_text.lower())
# self.WORDS = Counter(raw_words)

In [12]:
words = re.findall(r'\w+', text_misspel.lower())

In [13]:
edits = [checker.correction(word) for word in words]

In [14]:
words = [edit.edit for edit in edits]

In [15]:
words

['in',
 'toys',
 'rapidly',
 'advancing',
 'digital',
 'aet',
 'critical',
 'thinking',
 'has',
 'become',
 'more',
 'vital',
 'then',
 'ever',
 'with',
 'a',
 'flood',
 'of',
 'information',
 'available',
 'at',
 'our',
 'fingertips',
 'and',
 'an',
 'increasing',
 'number',
 'of',
 'platform',
 'for',
 'communication',
 'and',
 'expression',
 'the',
 'ability',
 'to',
 'think',
 'vertically',
 'is',
 'essential',
 'for',
 'navigatng',
 'this',
 'complex',
 'landscape',
 'in',
 'this',
 'partial',
 'we',
 'will',
 'explore',
 'the',
 'significance',
 'of',
 'critical',
 'thinking',
 'in',
 'the',
 'digital',
 'ague',
 'and',
 'discuss',
 'how',
 'it',
 'can',
 'be',
 'cultivated',
 'and',
 'applied',
 'in',
 'various',
 'aspects',
 'of',
 'ours',
 'gives',
 'critical',
 'thinning',
 'is',
 'the',
 'ability',
 'to',
 'objectiveley',
 'analyze',
 'and',
 'evaluate',
 'information',
 'ideas',
 'and',
 'arguments',
 'it',
 'involves',
 'questioning',
 'assumptions',
 'considering',
 'diff

In [16]:
text_correct

"in today's rapidly advancing digital age, critical thinking has become more vital than ever. with a flood of information available at our fingertips and an increasing number of platforms for communication and expression, the ability to think critically is essential for navigating this complex landscape. in this article, we will explore the significance of critical thinking in the digital age and discuss how it can be cultivated and applied in various aspects of our lives.\n\ncritical thinking is the ability to objectively analyze and evaluate information, ideas, and arguments. it involves questioning assumptions, considering different perspectives, and reaching well-reasoned conclusions. in the digital age, where misinformation and fake news abound, critical thinking acts as a shield against manipulation and deception. it allows us to discern between credible sources and unreliable ones, enabling us to make informed decisions and avoid being swayed by false narratives.\n\none of the c

In [17]:
" ".join(words)

'in toys rapidly advancing digital aet critical thinking has become more vital then ever with a flood of information available at our fingertips and an increasing number of platform for communication and expression the ability to think vertically is essential for navigatng this complex landscape in this partial we will explore the significance of critical thinking in the digital ague and discuss how it can be cultivated and applied in various aspects of ours gives critical thinning is the ability to objectiveley analyze and evaluate information ideas and arguments it involves questioning assumptions considering different prospective and reaching well reasoned conclusions in the digital age where misinforamtion and ukase news abound critical thinking acts as a shield against manipulation and deception it allows us to discern between tremble source and unrelaible ones enabling us to make informed decisions and avoid being swayed by false narratives one off the challenges we face in the d

In [18]:
text_misspel

'in todys rapidlly advancng digatal ae, critcial thinkng has beome mre vital then evar. with a flod of informaton availble at our ffingertips and an incresing numbr of platforsm for comminication and exoression, the ablity to thnk crtically is essentil for navigatng this complexx lanscape. in this artical, we wil expore the signifcanc of critcial thiking in the digiatl agge and disscuss how it can be cultiavted and applieed in varrious aspects of ourl ives.\n\ncritcal thikning is the abiliity to objectiveley analize and evaluatte informatoin, ideas, and arguements. it inolvves qestioning assumtions, considring difeerent prespectives, and reacghing well-reasoneed conclustions. in the digita age, wher misinforamtion and fkae news abuond, crtical thinkin acts as a shield against maniuplation and decption. it allows us to dissern betwen creible source and unrelaible ones, enablign us to make infomred decisiions and avoid being swayed by fals narraatives.\n\none off the challegnes we fac in